In [72]:
import pandas as pd
import os
from rutermextract import TermExtractor
import re
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

from pymystem3 import Mystem
m = Mystem()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

from numpy import array

#### Частотный словарь ####

In [34]:
df = pd.read_csv('./Freq2011/freqrnc2011.csv', sep='\t')
freq_dict = df[['Lemma','Freq(ipm)']]
sorted_freq = freq_dict.sort_values(by=['Freq(ipm)'])

In [8]:
len(freq_dict)

52138

In [9]:
sorted_freq.head()

,Lemma,Freq(ipm)
29836,перекачать,0.4
32918,помешанный,0.4
15847,касситерит,0.4
15850,кастаньета,0.4
32896,померанец,0.4


In [10]:
sorted_freq.tail()

,Lemma,Freq(ipm)
51957,я,12684.4
22019,на,15867.3
23328,не,18028.0
3710,в,31374.2
13824,и,35801.8


In [11]:
lemmas = list(freq_dict['Lemma'])
values = list(freq_dict['Freq(ipm)'])

dictionary = dict(zip(lemmas, values))

#### Биграммы ####

In [12]:
with open('bigrams.txt') as fl:
    bi = fl.read()
    
with open('trigrams.txt') as fl:
    tri = fl.read()
    
with open('tetragrams.txt') as fl:
    tetra = fl.read()
    

In [13]:
bigrams = bi.split('\n')
trigrams = tri.split('\n')
tetragrams = tetra.split('\n')

#### Аффиксы ####

In [14]:
with open('suf3.txt') as fl:
    s3 = fl.read()
    
with open('suf4.txt') as fl:
    s4 = fl.read()
    
with open('pref3.txt') as fl:
    p3 = fl.read()

with open('pref4.txt') as fl:
    p4 = fl.read()

In [15]:
suf3 = s3.split('\n')
suf4 = s4.split('\n')
pref3 = p3.split('\n')
pref4 = p4.split('\n')

#### Контекстные слова ####

In [16]:
with open('frequent_words.txt') as fl:
    cont = fl.read()

In [17]:
context = cont.split('\n')
print(context)

['изменение', 'анализ', 'количество', 'концентрация', 'образец', 'объем', 'понятие', 'амплитуда', 'действие', 'использование', 'использовать', 'исследование', 'метод', 'под названием', 'называется', 'называют', 'например', 'развитие', 'разновидность', 'рост', 'система', 'содержание', 'содержать', 'стадия', 'структура', 'термин', 'увеличение', 'уменьшение', 'уровень', 'формирование', 'формировать', 'эффект', 'дефект']


#### Кандидаты Rutermextractor ####

In [18]:
def get_term_candidates(file_path):
    
    with open(file_path) as fl:
        try:
            text = fl.read()
        except:
            print ("Error: " + file_path )
    
    terms = []
    term_extractor = TermExtractor()

    for term in term_extractor(text):
        terms.append(term.normalized)
    return terms

In [19]:
candidates = get_term_candidates('./terms_markup/Marked/2017_07_06_ksarelto-tromboz_.txt')

/opt/conda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [20]:
print(candidates)

['ривароксабан', 'препараты', 'препарат', 'фактор', 'тромбоз', 'мир', 'варфарин', 'тромбин', 'риск', 'кровь', 'война', 'indicator.ru', 'система свёртывания крови', 'фактор xa', 'побочные эффекты', 'крысиный яд', 'глубокие вена', 'работа', 'протромбин', 'врач', 'витамин', 'варфарина', 'фибрилляция предсердий', 'действующий вещество', 'фибрин', 'сша', 'рана', 'раз', 'организм', 'медицина', 'лекарство', 'кровотечение', 'инсульт', 'имя', 'друг', 'препараты старого поколения', 'всемирная организация здравоохранения', 'свёртывание крови', 'оторвавшийся тромб', 'молекула фибриногена', 'лёгочные сосуды', 'лёгочная эмболия', 'доказательная медицина', 'действующая форма', 'белые перчатки', 'фибриноген', 'фармацевты', 'тромб', 'точность', 'таблетка', 'сумма', 'справка', 'смерть', 'россия', 'риска', 'рак', 'процессы', 'профилактика', 'плацебо', 'пациенты', 'паутина', 'нея', 'нарушение', 'молекула', 'множество', 'мишень', 'кусочки', 'каскад', 'карманы', 'исследования', 'испытуемые', 'инсульты', 'ин

#### Предобработка #####

In [21]:
term_unigrams = []
term_bigrams = []
term_trigrams = []
term_multigrams = []

lem_unigrams = []
lem_bigrams = []
lem_trigrams = []
lem_multigrams = []

for term in candidates:
    if len(term.split()) == 1:
        term_unigrams.append(term.lower())
        lem_unigrams.append(m.lemmatize(term))
    elif len(term.split()) == 2:
        term_bigrams.append(term.lower())
        lem_bigrams.append(m.lemmatize(term))
    elif len(term.split()) == 3:
        term_trigrams.append(term.lower())
        lem_trigrams.append(m.lemmatize(term))
    else:
        term_multigrams.append(term.lower())
        lem_multigrams.append(m.lemmatize(term))

In [22]:
lem_uni = []

for u in lem_unigrams:
    lem_uni.append(u[0])

In [24]:
lem_bi = []
for b in lem_bigrams:
    lem_bi.append([b[0],b[2]])

In [26]:
lem_tri = []
for t in lem_trigrams:
    lem_tri.append([t[0],t[2],t[4]])

In [28]:
lem_multi = []

for mul in lem_multigrams:
    mul = [x for x in mul if x != ' ']
    mul = mul[:-1]
    lem_multi.append(mul)

#### Отсев кандидатов ####

In [32]:
final_unigrams = []
final_bigrams = []
final_trigrams = []
final_multigrams = []

In [75]:
#проверяем частотность
scores = []

for term in lem_uni:
    if term not in lemmas:
        scores.append(1)
    elif dictionary.get(term) < 100:
        scores.append(1)
    else:
        scores.append(0)
    

In [76]:
#проверяем префиксы
scores1 = []

for term in lem_uni:
    if term[-3:] in pref3 == True:
        scores1.append(1)
    else:
        scores1.append(0)

scores2 = []

for term in lem_uni:
    if term[-4:] in pref4 == True:
        scores2.append(1)
    else:
        scores2.append(0)

In [77]:
#проверяем суффиксы
scores3 = []

for term in lem_uni:
    if stemmer.stem(term)[:3] in suf3:
        scores3.append(1)
    else:
        scores3.append(0)

scores4 = []

for term in lem_uni:
    if stemmer.stem(term)[:4] in suf4:
        scores4.append(1)
    else:
        scores4.append(0)

In [78]:
s0 = array(scores)
s1 = array(scores1)
s2 = array(scores2)
s3 = array(scores3)
s4 = array(scores4)

In [79]:
total = s0 + s1 + s2 + s3 + s4

In [81]:
cand_scores = dict(zip(lem_uni,total))

In [82]:
print(cand_scores)

{'ривароксабан': 2, 'препарат': 1, 'фактор': 2, 'тромбоз': 1, 'мир': 0, 'варфарин': 2, 'тромбин': 1, 'риск': 1, 'кровь': 0, 'война': 0, 'indicator': 1, 'работа': 0, 'протромбин': 1, 'врач': 1, 'витамин': 1, 'варфарина': 2, 'фибрин': 1, 'сша': 1, 'рана': 1, 'раз': 0, 'организм': 1, 'медицина': 1, 'лекарство': 1, 'кровотечение': 1, 'инсульт': 1, 'имя': 0, 'друг': 0, 'фибриноген': 1, 'фармацевт': 1, 'тромб': 1, 'точность': 3, 'таблетка': 2, 'сумма': 0, 'справка': 1, 'смерть': 0, 'россия': 1, 'рак': 1, 'процесс': 0, 'профилактика': 2, 'плацебо': 1, 'пациент': 2, 'паутина': 1, 'она': 0, 'нарушение': 1, 'молекула': 2, 'множество': 0, 'мишень': 1, 'кусочек': 1, 'каскад': 1, 'карман': 0, 'исследование': 1, 'испытуемый': 1, 'инструкция': 1, 'закупорка': 1, 'действие': 0, 'группа': 0, 'герой': 1, 'воз': 1, 'внимание': 0, 'вещество': 1, 'белок': 2, 'база': 0, 'антикоагулянт': 2, 'яд': 1, 'эффективность': 1, 'эффект': 1, 'эритроцит': 1, 'эксперт': 1, 'экспериментатор': 1, 'эволюция': 1, 'шутка': 1

In [84]:
with  open('./terms_markup/Marked/2017_07_06_ksarelto-tromboz_.txt') as fl:
    text = fl.read()

reg_term = re.compile(r'&(.*?)!&')
terms = reg_term.findall(text)

In [85]:
uterms = []
for t in terms:
    if len(t.split()) == 1:
        uterms.append(m.lemmatize(t)[0])

In [87]:
marked_term = set(uterms)

In [90]:
for t in marked_term:
    if cand_scores.get(t) != 0:
        print(t, cand_scores.get(t))

протромбин 1
х 1
ривароксабан 2
варфарина 2
дженерик 1
варфарин 2
фибриноген 1
тромбин 1
тромбоз 1
антикоагулянт 2
тромбоцит 1
эритроцит 1
фибрин 1
